In [2]:
import rpy2.ipython
import warnings
warnings.filterwarnings('ignore')

%reload_ext rpy2.ipython
%reload_ext autoreload

import json
import pandas as pd
import os
from vega import VegaLite
from ipywidgets import widgets

from falx.chart import *

DATA_DIR = os.path.join("..", "benchmarks")

def render_ggplot2(vis):
    """Given a visualization (VisDesign object), 
        render it using ggplot"""
    
    ggplot_script = vis.to_ggplot2()
    print("\n".join(ggplot_script))
    for l in ggplot_script:
        %R -i l
        %R eval(parse(text=l))
    %R print(p)

def render_vegalite(vis):
    # Render a visualization using vegalite
    VegaLite(vis.to_vl_obj()).display()
    

        
def view_benchmarks(dirname):

    for fname in os.listdir(dirname):
        if not fname.endswith(".json"): continue
            
        if "003" not in fname: continue
    
        with open(os.path.join(dirname, fname), "r") as f:
            data = json.load(f)

        input_data = table_utils.load_and_clean_table(data["input_data"])
        vis = VisDesign.load_from_vegalite(data["vl_spec"], data["output_data"])
        try:
            render_vegalite(vis)
            render_ggplot2(vis)
        except:
            pass
#view_benchmarks(DATA_DIR)